# Chicago Transit Authority Ridership and Funding EDA


## Summary

Over the last few years, Chicago public transportation has lost ridership and funding in favor of private car ownership. Ever since the pandemic of 2020, ridership was massively cut and reliability of the service became questioned by the public. In 2024, Chicago sees ridership increasing, however, locals are finding the environment dramatically changed. The largest complaints of the CTA are:
1. Increase in violant crime on the train
2. Reliability and on-time delivery
3. Health or hygiene concerns of the trains due to homelessness or drug addiction

### Objective

(Review with Marcello)

With the limited funding provided, can we use public data to understand where financial support should be focused?


Can we determine if safety or reliability is the biggest impact to ridership recovery? (Crime vs. late busses/trains)


What "lines" have the biggest unrecovered population and what are the commonalities that can be remedied? 


## Data Sources

Data for ridership is publically available through the CTA developer site [here](https://data.cityofchicago.org/browse?q=ridership&sortBy=relevance)

Data for crime is posted by the city of Chicago [here](https://data.cityofchicago.org/Public-Safety/City-of-Chicago-Crime-Data/v9q9-3dm2)

## Data Dictionary and Glossary

### Crime

**Summary**
Reported crimes committed within the Chicago area and published by the Chicago Police Department. Data last refreshed as of 4th of April, 2024 00:00 (Central Standard Time / Chicago).

### Data Dictionary

**Case Number** - The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.

**Date** - Date when the incident occurred. this is sometimes a best estimate.

**Block** - The partially redacted address where the incident occurred, placing it on the same block as the actual address.

**IUCR** - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.

**Primary Type** - The primary description of the IUCR code.

**Description** - The secondary description of the IUCR code, a subcategory of the primary description.

**Location Description** - Description of the location where the incident occurred.

**Arrest** - Indicates whether an arrest was made.

**Domestic** - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.

**Beat** - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.

**District** - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.

**Ward** - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.

**Community Area** - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.

**FBI Code** - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS).See the Chicago Police Department listing of these classifications at https://gis.chicagopolice.org/pages/crime_details.

**X Coordinate** - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.

**Y Coordinate** - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.

**Year** - Year the incident occurred.

**Updated On** - Date and time the record was last updated. 

**Latitude** - The latitude of the location where incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.

**Longitude** - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.

**Location** - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.

### L Stop Daily Rides

**Summary**
This list shows daily totals of ridership, by station entry, for each 'L' station dating back to 2001. Dataset shows entries at all turnstiles, combined, for each station. Daytypes are as follows: W=Weekday, A=Saturday, U=Sunday/Holiday

### Data Dictionary

**station_id** - City ID for station

**stationname** - Street Intersection/Name of the station

**date** - Day of Year

**daytype** - W=Weekdays, A=Saturday, U=Sunday/Holiday

**rides** - Total rides by day for each station

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import polars


pd.options.display.max_columns = None

In [74]:
crimes = pd.read_csv("crimes.csv")
l_stop_daily_rides = pd.read_csv("l_stop_daily_rides.csv")
comm_area = pd.read_csv("CommAreas.csv")

# Understanding the Data

A review of the data is necessary to understand where unnecessary features are listed or data that needs to be sanitized before it becomes useable. Below we see each dataset reviewed for feature data types, and null/NaN values. Any features where nulls are highly present will require adjustments or dropping entirely. 

In [75]:
l_stop_daily_rides.dtypes

station_id      int64
stationname    object
date           object
daytype        object
rides           int64
dtype: object

In [76]:
crimes.dtypes

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

## Updating the Column Names

It is best practice to update column names into snake case

In [83]:
c_cols = crimes.columns
l_cols = l_stop_daily_rides.columns
com_cols = comm_area.columns

def minimize_cols(cols: list[str]) -> list:
    new_cols = []
    for i in range(len(cols)):
        updated_col = cols[i].lower()
        updated_col = updated_col.replace(" ","_")
        new_cols.append(updated_col)
    return new_cols



crimes.rename(dict(zip(c_cols,minimize_cols(c_cols))), inplace=True)
l_stop_daily_rides.rename(dict(zip(l_cols,minimize_cols(l_cols))),inplace=True)
comm_area.rename(dict(zip(com_cols,minimize_cols(com_cols))), inplace=True)



ValueError: Length mismatch: Expected axis has 22 elements, new values have 8036727 elements

In [78]:
crime_nulls = {}
for col in crimes.columns:
    crime_nulls[col] = len(crimes[crimes[col].isna()])
crime_nulls


{'ID': 0,
 'Case Number': 0,
 'Date': 0,
 'Block': 0,
 'IUCR': 0,
 'Primary Type': 0,
 'Description': 0,
 'Location Description': 12851,
 'Arrest': 0,
 'Domestic': 0,
 'Beat': 0,
 'District': 47,
 'Ward': 614851,
 'Community Area': 613475,
 'FBI Code': 0,
 'X Coordinate': 88559,
 'Y Coordinate': 88559,
 'Year': 0,
 'Updated On': 0,
 'Latitude': 88559,
 'Longitude': 88559,
 'Location': 88559}

In [79]:
l_stop_nulls = {}
for col in l_stop_daily_rides.columns:
    l_stop_nulls[col] = len(l_stop_daily_rides[l_stop_daily_rides[col].isna()])
l_stop_nulls

{'station_id': 0, 'stationname': 0, 'date': 0, 'daytype': 0, 'rides': 0}

## Missing Data

We can see that no values are lost for the CTA ridership data, however, our crime data is missing data in the coordinate feature. 



Talk with Marcello:
* Should I spend the time to apply the coordinates by using google for each L stop in Chicago? 
* I think coordinates are helpful when considering the proximity to the CTA stops but it doesn't directly impact the crimes related to riders on the CTA
* Conversions of data
    * Dates are listed as objects in crimes, for grouping and sorting i believe this should be updated



----------

Next Steps:
1. Replace Community Area with name from commarea csv
2. Drop X,Y, drop missing values for Longitude and Lat, Location features
3. Convert dates to datetime
4. Missing values for objects: 'na' or not available
5. Review Beats and create new column for stops within the beat
6. 

In [80]:
l_stop_daily_rides['date'].head()

0    12/22/2017
1    12/18/2017
2    12/02/2017
3    12/19/2017
4    12/03/2017
Name: date, dtype: object

In [82]:
crimes.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [ ]:
l_stop_daily_rides.head(10)

,station_id,stationname,date,daytype,rides
0,41280,Jefferson Park,12/22/2017,W,6104
1,41000,Cermak-Chinatown,12/18/2017,W,3636
2,40280,Central-Lake,12/02/2017,A,1270
3,40140,Dempster-Skokie,12/19/2017,W,1759
4,40690,Dempster,12/03/2017,U,499
5,41660,Lake/State,12/30/2017,A,8615
6,40180,Oak Park-Forest Park,12/17/2017,U,442
7,40250,Kedzie-Homan-Forest Park,12/02/2017,A,1353
8,40120,35th/Archer,12/07/2017,W,3353
9,41420,Addison-North Main,12/19/2017,W,6034


In [ ]:
comm_area.head()

,the_geom,PERIMETER,AREA,COMAREA_,COMAREA_ID,AREA_NUMBE,COMMUNITY,AREA_NUM_1,SHAPE_AREA,SHAPE_LEN,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187,Unnamed: 188,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,Unnamed: 222,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Unnamed: 256,Unnamed: 257,Unnamed: 258,Unnamed: 259,Unnamed: 260,Unnamed: 261,Unnamed: 262,Unnamed: 263,Unnamed: 264,Unnamed: 265,Unnamed: 266,Unnamed: 267,Unnamed: 268,Unnamed: 269,Unnamed: 270,Unnamed: 271,Unnamed: 272,Unnamed: 273,Unnamed: 274,Unnamed: 275,Unnamed: 276,Unnamed: 277,Unnamed: 278,Unnamed: 279,Unnamed: 280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292,Unnamed: 293,Unnamed: 294,Unnamed: 295,Unnamed: 296,Unnamed: 297,Unnamed: 298,Unnamed: 299,Unnamed: 300,Unnamed: 301,Unnamed: 302,Unnamed: 303,Unnamed: 304,Unnamed: 305,Unnamed: 306,Unnamed: 307,Unnamed: 308,Unnamed: 309,Unnamed: 310,Unnamed: 311,Unnamed: 312,Unnamed: 313,Unnamed: 314,Unnamed: 315,Unnamed: 316,U